# Exercice 1 Réalisation d’un outil pour dessiner des réseaux

Vous devez réaliser un logiciel permettant de dessiner des architectures réseau de façon
simple et élégante. 
Le cahier des charges imposé par le client indique que votre logiciel
doit permettre de représenter :
- des switchs
- des clients (PC ou téléphone)
- des routeurs

Les différents éléments peuvent être présents de multiple fois dans l’interface. 
La création d’un nouvel item pourra se faire par un raccourci clavier ( C client, S switch, R routeur, par exemple ) ou par toolbar / menu, et il doit être possible de déplacer ou d’effacer les items à la souris.

Un clic droit sur un item doit permettre d’en modifier les propriétés :
- Nom
- Icône

Enfin, il doit être possible de dessiner des traits sur le schémas afin de représenter les liens réseau.

In [ ]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        # 1) 
        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

# 1) 
root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()

J'ai dans mon premier commit, créer une classe qui permettra d'instancier les objets de l'application car il doivent être présents plusieurs fois en même temps dans la fenêtre.


A la fin de mon programme, je crée une fenêtre qui s'appelle root et à laquelle je créer une instance de la classe "WishPacketTracer".


Et donc le programme va créer un canvas basique de 800x600 qui va s'étendre càd prendre la place dans tout son espace (en l'occurence la fenêtre) et doit s'élargir horizontalement et verticalement à la fois avec le "pack(expand pour l'extension et fill pour l'élargissement)".

<img src="./img/i1.png">

Puis j'ai ajouté à mon code, une barre d'outils dans laquelle, j'ai au préalable chargé mes 3 images (pc,routeur et switch) grâce à PIL et je les redimensionne et je les ai rendues cliquables à l'aide de la métode "Button" dans ma barre d'outils et à l'appui du bouton (donc de l'image), une image va être créée dans mon canvas à l'aide de la méthode "create_image".

In [ ]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)
        
        # Barre d'outils
        self.toolbar = tk.Frame(root)
        self.toolbar.pack(side=tk.LEFT, fill=tk.Y)

        # Boutons de la barre d'outils (utilisant des images redimensionnées avec PIL)
        self.image_router = self.load_and_resize_image("./img/Router.png", 50, 50)
        self.button_router = tk.Button(self.toolbar, image=self.image_router, command=self.create_router)
        self.button_router.pack(side=tk.LEFT)

        self.image_switch = self.load_and_resize_image("./img/switch.png", 50, 50)
        self.button_switch = tk.Button(self.toolbar, image=self.image_switch, command=self.create_switch)
        self.button_switch.pack(side=tk.LEFT)

        self.image_pc = self.load_and_resize_image("./img/pc.png", 50, 50)
        self.button_client = tk.Button(self.toolbar, image=self.image_pc, command=self.create_client)
        self.button_client.pack(side=tk.LEFT)

    def create_client(self):
        item = self.canvas.create_image(100, 100, image=self.image_pc, tags="client")
    def create_switch(self):
        item = self.canvas.create_image(200, 200, image=self.image_switch, tags="switch")
    
    def create_router(self):
        item = self.canvas.create_image(300, 300, image=self.image_router, tags="router")
    
   
    def load_and_resize_image(self, filename, width, height):
        image = Image.open(filename)
        image = image.resize((width, height))
        photo = ImageTk.PhotoImage(image)
        return photo

root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()

<img src="./img/i2.png">

Puis, j'ai implémenter la possibilité de pouvoir déplacer les différents items dans mon canvas à l'aide de deux écouteurs d'évènements qui vont se succéder : 
1. "Button-1" Qui va écouter au clic gauche sur mon canvas et va déclencher une fonction "left_click" (clic gauche)


- Et cette fonction, prend en arguments "self" pour l'instance de la classe et "event" (pour le clic càd les coordonnées en abscisses et ordonnées du clic).


- Puis j'affecte à une variable item le résultat de la méthode "find_closest" qui va rechercher l'élément le plus proche des coordonnées du clic (à l'aide des arguments "event.x" pour la coord en abscisse du clic et "event.y" pour la coord en ordonnée du clic).


- Ensuite, une vérification pour savoir s'il y a un élément à l'emplacement du clic et si oui, on défini à une variable instanciée "self.current_item", la 1ère valeur et unique de item.

2. "B1-Motion" Qui va écouter au clic gauche enfoncé sur mon canvas et déclencher une fonction "drag_item" prenant en argument en + de l'instance, "event" pour récupérer les coordonnées du curseur enfoncé.


- Il vérifie d'abord s'il y a une valeur dans "self.current_item" et donc en ayant appuyé avant de pouvoir rester enfoncé, on a déclencher la fonction "left_click" qui a attribué une valeur à "self.current_item".


- Il récupère alors les coordonées actuelles de la souris en abscisses / ordonnées (event.x,event.y) puis va à chacun soustraire les coordonnées actuelles de l'élément sélectionné en abscisses "self.canvas.coords(self.current_item)[0]" et puis en ordonnées "self.canvas.coords(self.current_item)[1]"


- Et la différence des coords actuelles de la souris moins les coords actuelles de l'élément sélectionner nous donne la distance parcourue par la souris avec en abscisses "dx" et en ordonnées "dy".

Ainsi, la méthode "move" permet de déplacer un élément (ici, "self.current_item" donc l'élément sélectionné) de "dx" pixels (une distance horizontale) et de "dy" pixels (une distance verticale).
C'est comme cela, qu'une image peut être "glissée-déposée" (drag-and-drop).

In [ ]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        # Barre d'outils
        self.toolbar = tk.Frame(root)
        self.toolbar.pack(side=tk.LEFT, fill=tk.Y)

        # Boutons de la barre d'outils (utilisant des images redimensionnées avec PIL)
        self.image_router = self.load_and_resize_image("./img/Router.png", 50, 50)
        self.button_router = tk.Button(self.toolbar, image=self.image_router, command=self.create_router)
        self.button_router.pack(side=tk.LEFT)

        self.image_switch = self.load_and_resize_image("./img/switch.png", 50, 50)
        self.button_switch = tk.Button(self.toolbar, image=self.image_switch, command=self.create_switch)
        self.button_switch.pack(side=tk.LEFT)

        self.image_pc = self.load_and_resize_image("./img/pc.png", 50, 50)
        self.button_client = tk.Button(self.toolbar, image=self.image_pc, command=self.create_client)
        self.button_client.pack(side=tk.LEFT)

        self.canvas.bind("<Button-1>", self.left_click)
        self.canvas.bind("<B1-Motion>", self.drag_item)

    def create_client(self):
        item = self.canvas.create_image(100, 100, image=self.image_pc, tags="client")

    def create_switch(self):
        item = self.canvas.create_image(200, 200, image=self.image_switch, tags="switch")
    def create_router(self):
        item = self.canvas.create_image(300, 300, image=self.image_router, tags="router")

    def left_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]

    def drag_item(self, event):
        if self.current_item:
            dx = event.x - self.canvas.coords(self.current_item)[0]
            dy = event.y - self.canvas.coords(self.current_item)[1]
            self.canvas.move(self.current_item, dx, dy)

    def load_and_resize_image(self, filename, width, height):
        image = Image.open(filename)
        image = image.resize((width, height))
        photo = ImageTk.PhotoImage(image)
        return photo

root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()


Position des éléments de base : 
<img src="./img/i2.png">

Position des élements après déplacement : 
<img src="./img/i3.png">

Il faut également pouvoir créer routeurs / switchs / pc à l'aide d'appuis sur le clavier et c'est pourquoi, j'ai ajouté un écouteur d'évènement "Key" qui va déclencher la fonction "key_pressed".


"key_pressed" est une fonction prenant en argument "event", qui signifie qu'une touche clavier a été tapée puis on y applique la méthode "char" pour retrouver la touche clavier qui a déclenché l'event (ex: on appuie sur la touche "A", "event.char" contiendra "a") et enfin on y applique la méthode "upper" pour s'assurer que la chapine de caractère sera toujours en majuscules et on affecte le résultat à une variable "key".


Ainsi "key" sera une chaine de caractère contenant la touche tapée (ex: on appuie sur la touche "S", key contiendra "S").


Puis on compare "key" à 3 chaine de caractères "C" ou "S" ou "R" et qui vont créer à chacun un pc ou un switch ou un routeur. Si une autre touche a été tapée que ces 3 là alors rien ne se passera.

In [ ]:
import tkinter as tk
from tkinter import simpledialog
from PIL import Image, ImageTk

class WishPacketTracer:
    def __init__(self, root):
        self.root = root
        self.root.title("Wish Packet Tracer")

        self.canvas = tk.Canvas(root, bg="ivory", width=800, height=600)
        self.canvas.pack(expand=tk.YES, fill=tk.BOTH)

        # Barre d'outils
        self.toolbar = tk.Frame(root)
        self.toolbar.pack(side=tk.LEFT, fill=tk.Y)

        # Boutons de la barre d'outils (utilisant des images redimensionnées avec PIL)
        self.image_router = self.load_and_resize_image("./img/Router.png", 50, 50)
        self.button_router = tk.Button(self.toolbar, image=self.image_router, command=self.create_router)
        self.button_router.pack(side=tk.LEFT)

        self.image_switch = self.load_and_resize_image("./img/switch.png", 50, 50)
        self.button_switch = tk.Button(self.toolbar, image=self.image_switch, command=self.create_switch)
        self.button_switch.pack(side=tk.LEFT)

        self.image_pc = self.load_and_resize_image("./img/pc.png", 50, 50)
        self.button_client = tk.Button(self.toolbar, image=self.image_pc, command=self.create_client)
        self.button_client.pack(side=tk.LEFT)

        # Evènements souris
        self.canvas.bind("<Button-1>", self.left_click)
        self.canvas.bind("<B1-Motion>", self.drag_item)
        # Evènements clavier
        self.root.bind("<Key>", self.key_pressed)

    def create_client(self):
        item = self.canvas.create_image(100, 100, image=self.image_pc, tags="client")

    def create_switch(self):
        item = self.canvas.create_image(200, 200, image=self.image_switch, tags="switch")
    def create_router(self):
        item = self.canvas.create_image(300, 300, image=self.image_router, tags="router")

    def left_click(self, event):
        item = self.canvas.find_closest(event.x, event.y)
        if item:
            self.current_item = item[0]

    def drag_item(self, event):
        if self.current_item:
            dx = event.x - self.canvas.coords(self.current_item)[0]
            dy = event.y - self.canvas.coords(self.current_item)[1]
            self.canvas.move(self.current_item, dx, dy)

    def load_and_resize_image(self, filename, width, height):
        image = Image.open(filename)
        image = image.resize((width, height))
        photo = ImageTk.PhotoImage(image)
        return photo

    def key_pressed(self, event):
        key = event.char.upper()
        if key == "C":
            self.create_client()
        elif key == "S":
            self.create_switch()
        elif key == "R":
            self.create_router()

root = tk.Tk()
app = WishPacketTracer(root)
root.mainloop()


# Exercice 2 Des traits bien droits !
Faites en sorte que les lignes ne puissent être que horizontales ou verticales lorsque
l’utilisateur maintient la touche CTRL enfoncée.


# Exercice 3 Un client exigeant ...
Le client, satisfait par votre travail précédant, demande à ce que vous alliez plus loin. Il
souhaite maintenant que les items aient des ports sur lesquels votre souris sera aimantée
lorsque vous dessinez des traits. Le nombre de ports des routeurs et des switchs sera
paramétrable de 1 à 4. Les clients, eux, auront toujours un seul port